<a href="https://colab.research.google.com/github/LEESUSUSUSU/Image_based_question_answering_AI_project/blob/Suyeon/DL_project_BaselineReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os  #os 운영체제와 상호 작용 하기위 한 기능
import pandas as pd #판다스 데이터 조작을 위함

import torch  # 토치 높은 텐서 연산을 지원한다.
import torch.optim as optim # 최적화 알고리즘을 구현하는 모듈 sgd, adam 등 다양한 최적화
import torch.nn as nn # 딥러닝 모델을 구성하는데 필요한 다양한 레이어와 함수를 제공한다.
from torch.utils.data import Dataset, DataLoader #데이터를 배치 단위로 불러오는 반복자 역활을 합니다.


import torchvision.models as models # 사전 훈련된 모델들을 포함하고 있어, 이미지분류, 객체감지 드의 컴퓨터 비전 태스크를 쉽게 적용할 수 있다.
from torchvision import transforms # 이미지 전처리를 위한 다양한 기능을 제공하고 이미지를 텐서로 변화하거나 크기,조정,정규화,데이터 증강 등의 작업을 수행 할 수 있다.
from PIL import Image # 이미를 불러오고, 저장,처리 등을 제공하고 위의 torchvision과 함께 사용되어 데이터를 취급할때 유용하다.

from transformers import GPT2Tokenizer, GPT2Model # 자연어처리 nlp를 위한 사전 훈련된 모델을 제공한다.gpt-2를 제공

from tqdm.auto import tqdm #반복 작업의 진행 상태를 시각적으로 보여주는 라이브러리


In [5]:
class VQADataset(Dataset): # 이미지와 관련된 질문에 대한 답변을 학습하기 위해 사용되는 데이터

    def __init__(self, df, tokenizer, transform, img_path, is_test=False):
        self.df = df # df 데이터 프레임
        self.tokenizer = tokenizer # 토크나이저
        self.transform = transform # 이미지 변환기
        self.img_path = img_path # 이미지 경로
        self.is_test = is_test # 데스트 데이터 셋 여부


    def __len__(self):  # 데이터셋에 포함된 샘플의 총 개수를 변환
        return len(self.df) # 데이터 길이를 제공

    def __getitem__(self, idx):
        row = self.df.iloc[idx] # 특정 인덱스 샘플을 데이터셋에서 불러오고

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg') # 이미지 이름
        image = Image.open(img_name).convert('RGB') # 가져온 이미지를 rgb형식으로 불러와 저장
        image = self.transform(image) #이미지를 변환 저장

        #gpt-2 를 이용하기 때문에 인코더를 디코더는 하지 않는다.
        question = row['question'] # question을  토크나이징 해서 gpt-2 모델에 적합한 형태로 변환
        question = self.tokenizer.encode_plus( # 토큰라이즈.인코더 안에 세부 내용을 정리해준다.
            question, # 질문
            truncation=True, # 토큰의 길이가 max_length 보다 길면 일관성을 유지 하기 위해 필요함
            add_special_tokens=True, # 시작토큰과 종료 토근 텍스트에 자동으로 추가 해서 시작과 끝을 인식하는데 도움을 준다
            max_length=32, # 토큰화된 텍스트의 최대 길이를 지정한다.
            padding='max_length', # max_length 의 갖을 수 있도록 하는데 이는 배치 처리시 모든 샘플의 크기를 일정하게 유지
            return_attention_mask=True, #주의력 마스크 실제 데이터와 패딩을 구분할 수 있게 해주고 트랜스포머 기반 모델중에서 가장 중요 하다.
            return_tensors='pt', #반환되는 토큰화된 텍스트와 주의력 마스크를 토치 텐서로 반환하여 지정한다.
        )

        if not self.is_test:  # 테스트 모델이 아닐때
            answer = row['answer'] # 답변처리
            answer = self.tokenizer.encode_plus(
                answer,
                max_length=32,
                padding='max_length',
                truncation=True,
                return_tensors='pt')
            return {
                'image': image.squeeze(),
                'question': question['input_ids'].squeeze(),
                'answer': answer['input_ids'].squeeze()
            }
        else: # 테스트 모델일때
            return {
                'image': image,
                'question': question['input_ids'].squeeze(),
            }

# Model

In [3]:
class VQAModel(nn.Module):
    def __init__(self, vocab_size):  #모델을 초기화 함수 입니다. 모델에 필요한 레이어와 변수를 정의
        super(VQAModel, self).__init__() # 모델이 예측해야 하는 단어 집합의 크기
        self.vocab_size = vocab_size  # 출력데이터에 사용

        self.resnet = models.resnet50(pretrained=True) #사전 훈련된 모델을 불러와 이미지 특성을 추출
        self.gpt2 = GPT2Model.from_pretrained('gpt2') # gpt2 모델을 불러옴
        self.gpt2.resize_token_embeddings(vocab_size) # 추가한 [PAD] 토큰 반영

        combined_features_size = 1000 + self.gpt2.config.hidden_size # resnet이 1000 차원임 그래서 최종 벡터의 크기를 줘서 이미지 특성을 추출함
        self.classifier = nn.Linear(combined_features_size, vocab_size) # 결합된 이미지 와 텍스트 특성을 사용해 최종 답변 예측

    def forward(self, images, question): # 실제 데이터 처리
        image_features = self.resnet(images) # 이미지 배치
        image_features = image_features.view(image_features.size(0),-1) # 토큰화된 형태 -1 를 사용해서 남어지 원소의 배치들 분배 하고 순차적으로 나열

        outputs = self.gpt2(question) # 질문특성 추출
        output_features = outputs.last_hidden_state # [batch, sequence, hidden] 으닉 상태를 질문 득성

        # 이미지 특성을 gpt-2 모델의 출력과 같은 차원으로 확장 해줌
        image_features = image_features.unsqueeze(1).expand(-1, output_features.size(1),-1) # [batch, sequence, 1000]

        # 결합하여 최종 특성 벡터를 생성
        combined = torch.cat([image_features, output_features], dim=-1) # [batch, sequence, 1000+hidden]
        # 결합된 특성 벡터를 분류기에 통과시켜 각 단어에 대한 확률 분포 예측
        output = self.classifier(combined) # [batch, vocab_size]
        return output

# Dataloader

In [6]:
# 데이터 불러오기
train_df = pd.read_csv('/content/drive/MyDrive/zerobase/project_DL/train.csv') #train.csv 불러오기
test_df = pd.read_csv('/content/drive/MyDrive/zerobase/project_DL/test.csv') #test.csv  불러오기 등등
sample_submission = pd.read_csv('/content/drive/MyDrive/zerobase/project_DL/sample_submission.csv') # 무튼 불러오기
train_img_path = '/content/drive/MyDrive/zerobase/project_DL/image/train'
test_img_path = '/content/drive/MyDrive/zerobase/project_DL/image/test'

# dataset & dataloader
tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # 훈련된 톤큰나이저 를 불러온다.
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # 토큰 추가 시작 끝
vocab_size = len(tokenizer) # 전체 단어 집합 크기를 변수에 저장

transform = transforms.Compose([ # 데이터 전처리 변환
    transforms.Resize((224, 224)), # 이미지 데이터 전처리를 하기 위한 트렌스포머 resziesms 크기 조정
    transforms.ToTensor(), # 이미지를 텐서로 수정
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 픽셀 값을 정규화
])

#데이터셋 및 데이터로더 설정
train_dataset = VQADataset(train_df, tokenizer, transform, train_img_path, is_test=False) #위에 만든 클래스를 사용함
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 생성된 데이터셋 기반으로 로드 함 배친는 64

# Train & Inference

In [8]:
def train(model, loader, optimizer, criterion):  # 모델을 학습 하고 손실을 계산 하고 모델의 가중치를 업데이트
    model.train() # 학습할 딥딥러닝 모델
    total_loss = 0

    #매개변수
    for data in tqdm(loader, total=len(loader)): #tqem을 통해 퍼센트 확인 쌉가능
        images = data['image'].to(device) #이미지를 deviceㄹ(지정된 gpu) 이동
        question = data['question'].to(device) # 동일
        answer = data['answer'].to(device)

        optimizer.zero_grad() # 각 배치의 시작에 옵티마이저의 기울기를 0으로 초기화 역전파 과정에서 계산된 기울기가 누적을 방지

        outputs = model(images, question) # 모델에 이미지와 질문을 입력하여 예측결과 얻기

        # output: [batch, sequence, vocab], answer : [batch, sequence]
        loss = criterion(outputs.view(-1, outputs.size(-1)), answer.view(-1)) #view 메소드는 테너의 형태를 변경하여 손실함수에 적합
        total_loss += loss.item()

        loss.backward() # 손실에 대한 역전파 수행해서 모델의 각 가중치에 대한 손실의 기울기를 계산
        optimizer.step() # 계산된 기울기를 사용해서 모델의 가중치 업데이트 해줌


    avg_loss = total_loss / len(loader) # 에폭 동안의 평균 손실 계산
    return avg_loss # 리턴 값은 에폭 동안의 평균 손실값

In [9]:
def inference(model, loader):  # 데이터 추론 할거
    model.eval() # 모델을 평가 모드로 설정하는 것임
    preds = [] # 그래서 preds 임
    with torch.no_grad(): #기울기 계산할 필요 없어서 비활성화 후 메모리 사용량 줄이고 계산 속도를  향상
        for data in tqdm(loader, total=len(loader)):  # 사실 위랑 비슷함
            images = data['image'].to(device)
            question = data['question'].to(device)

            outputs = model(images, question) # [batch, sequence, vocab]

            _, pred = torch.max(outputs, dim=2) # values, indices = _, pred
            preds.extend(pred.cpu().numpy()) # cpu를 사용하는것은 넘파이를 사용을 용이 하기 위해서 그렇게 하는것

    return preds # 예측 결과 수행한값 반환 하는 것.

#Run!

In [ ]:
# # device
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # cuda가 사용가능하면 cuda 사용
# print(f"current device is {device}") # 쿠다가 사용이 가능하면 current 뜸

# # Model
# model = VQAModel(vocab_size).to(device) # modle 에 이제 적용해서 할거임

# # Criterion and Optimizer
# criterion = nn.CrossEntropyLoss() # 손실 함수로 저거 사용 분류 문제에서 주로 사용 하는 녀석
# optimizer = optim.AdamW(model.parameters(), lr=5e-5) #최적화 알고리즘의 변형이고 원래는 adamW에서 변형됐다 함
# #가중치 감소를 다루는 방식에서 차이가 있는 모델이라고 하는데 학습률은 엄청 작게함

# # Training loop
# for epoch in range(1): # 1 에폭 동안 수행하는 거고 필요에 따라서 언제든 변경 가능 가능...
#     avg_loss = train(model, train_loader, optimizer, criterion)
#     print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")

#     ## 끝나간다 나는 이제 잘 수 있다.

In [15]:
#예외처리값

def train(model, loader, optimizer, criterion):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0

    for data in tqdm(loader, total=len(loader)):
        try:
            images = data['image'].to(device)  # 이미지 데이터를 디바이스로 이동
            question = data['question'].to(device)
            answer = data['answer'].to(device)

            optimizer.zero_grad()

            outputs = model(images, question)  # 모델의 예측 결과를 얻음

            # 손실 계산
            loss = criterion(outputs.view(-1, outputs.size(-1)), answer.view(-1))
            total_loss += loss.item()

            # 역전파 및 옵티마이저 스텝
            loss.backward()
            optimizer.step()

        except FileNotFoundError as e:
            print(f"Warning: Skipping missing file {e.filename}")
            continue  # 해당 배치를 건너뛰고 다음 배치로 이동

    avg_loss = total_loss / len(loader)
    return avg_loss


# Post-Processing

In [ ]:
# 결과를 후처리 하는 가정임 불필요한 패딩 토큰 제거하고

# Dataset & DataLoader
test_dataset = VQADataset(test_df, tokenizer, transform, test_img_path, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# inference
preds = inference(model, test_loader)

no_pad_output = []
for pred in preds:
    output = pred[pred != 50257] # [PAD] token 제외
    no_pad_output.append(tokenizer.decode(output).strip()) # 토큰 id -> 토큰

In [ ]:
# 모델의 state_dict 저장하기
torch.save(model.state_dict(), 'Baseline_model.pth')



#Submission

In [ ]:
sample_submission['answer'] = no_pad_output
sample_submission.to_csv('/content/drive/MyDrive/zerobase/project_DL/submission_baselinere.csv', index=False)

In [ ]:
solution = pd.read_csv('/content/drive/MyDrive/zerobase/project_DL/submission_저장/solution.csv')